# Initialization

In [1]:
!pip install boto3 progressbar2 sfdmap GPUtil

  Using cached boto3-1.20.27-py3-none-any.whl (131 kB)
  Using cached progressbar2-3.55.0-py2.py3-none-any.whl (26 kB)
  Using cached sfdmap-0.1.1-py3-none-any.whl
  Using cached GPUtil-1.4.0-py3-none-any.whl
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached s3transfer-0.5.0-py3-none-any.whl (79 kB)
  Using cached botocore-1.23.27-py3-none-any.whl (8.5 MB)
  Using cached python_utils-2.7.1-py2.py3-none-any.whl (14 kB)


In [2]:
import GPUtil
print('GPUs:\n{0}'.format('\n'.join(['('+str(i+1)+')\t'+gpu.name for i,gpu in enumerate(GPUtil.getGPUs())])))

GPUs:
(1)	Tesla V100-SXM2-32GB


In [3]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
from matplotlib import pyplot as plt
import matplotlib
import boto3

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'git')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3.nautilus.optiputer.net'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'workdir3')
s3_saves_dir_path = os.path.join(s3_work_dir_path , 'model_saves')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_data_ver_dir_path = os.path.join(s3_data_dir_path,'HighSNR_12K_V1')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig

# Load the Data

In [4]:
load_RF_name = 'simple___2021_11_27___22_09_00___standard_RF_max_depth_10'
s3_load_dir_path = os.path.join(s3_saves_dir_path, 'RF', load_RF_name)
print('loading from folder (S3): {0}'.format(s3_load_dir_path))
X = from_s3_npy(s3_client, bucket_name, os.path.join(s3_load_dir_path, 'X.npy'))
sim_mat = from_s3_npy(s3_client, bucket_name, os.path.join(s3_load_dir_path, 'sim_mat.npy'))
dis_mat = 1 - sim_mat

loading from folder (S3): almogh/workdir3/model_saves/RF/simple___2021_11_27___22_09_00___standard_RF_max_depth_10
loading from uri: s3://tau-astro/almogh/workdir3/model_saves/RF/simple___2021_11_27___22_09_00___standard_RF_max_depth_10/X.npy
loading from uri: s3://tau-astro/almogh/workdir3/model_saves/RF/simple___2021_11_27___22_09_00___standard_RF_max_depth_10/sim_mat.npy


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, I_train, I_test = train_test_split(X, np.arange(X.shape[0]), train_size=9000, random_state=seed)

batch_size = 128

from NN import DistillationDataGenerator
train_gen = DistillationDataGenerator(X_train, dis_mat[I_train,:][:,I_train], shuffle=False, seed=seed, batch_size=batch_size, full_epoch=True)
val_gen = DistillationDataGenerator(X_val, dis_mat[I_test,:][:,I_test], shuffle=False, seed=seed, batch_size=batch_size, full_epoch=True)

In [6]:
from s3 import s3_load_TF_model
from NN import DistanceLayer
siamese_model = s3_load_TF_model(s3_client,
                                 bucket_name=bucket_name,
                                 #path_in_bucket='almogh/workdir3/model_saves/NN/simple___2021_12_06___21_36_21___Kernels_31_Filters_64_32_16_8_4_Hiddens_512_128_tanh/model',
                                 path_in_bucket='almogh/workdir3/model_saves/NN/simple___2021_12_13___11_16_51___Kernels_31_Filters_64_32_16_8_4_Hiddens_512_128_tanh_SNR_6dB_40dB/model',
                                 model_name='model',
                                 custom_objects={'DistanceLayer': DistanceLayer})

2022-01-04 09:19:42.510025: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-04 09:19:43.188593: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30996 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0


In [7]:
siamese_model.summary()

Model: "siamese_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None,)                   655372    
Total params: 655,374
Trainable params: 653,844
Non-trainable params: 1,530
_________________________________________________________________


In [8]:
siamese_model.siamese_network.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_input (InputLayer)        [(None, 7800)]       0                                            
__________________________________________________________________________________________________
second_input (InputLayer)       [(None, 7800)]       0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 128)          655372      first_input[0][0]                
                                                                 second_input[0][0]               
__________________________________________________________________________________________________
distance_layer (DistanceLayer)  (None,)              0           model[0][0]                

In [9]:
Z = siamese_model.predict(train_gen, verbose=1)

2022-01-04 09:19:47.075236: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-04 09:19:47.948824: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101


316442/316442 [==============================] - 7063s 22ms/step


In [10]:
from s3 import to_s3_npy
#to_s3_npy(Z, s3_client, bucket_name, 'almogh/workdir3/model_saves/NN/simple___2021_12_06___21_36_21___Kernels_31_Filters_64_32_16_8_4_Hiddens_512_128_tanh/Z.npy')
to_s3_npy(Z, s3_client, bucket_name, 'almogh/workdir3/model_saves/NN/simple___2021_12_13___11_16_51___Kernels_31_Filters_64_32_16_8_4_Hiddens_512_128_tanh_SNR_6dB_40dB/Z.npy')

saving to uri: s3://tau-astro/almogh/workdir3/model_saves/NN/simple___2021_12_13___11_16_51___Kernels_31_Filters_64_32_16_8_4_Hiddens_512_128_tanh_SNR_6dB_40dB/Z.npy


True